In [1]:
from azureml.core import Workspace


ws = Workspace(subscription_id="",
               resource_group="",
               workspace_name="")

In [2]:
import urllib.request
from azureml.core.model import Model

# Download model
# urllib.request.urlretrieve("https://aka.ms/bidaf-9-model", "model.onnx")

# Register model
# model = Model.register(ws, model_name="bidaf_onnx", model_path="./model.onnx")

model = Model.register(ws, model_name="flair_sample_model", model_path="D:\\NER\\flair_sample_model.pt")


Registering model flair_sample_model


In [3]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig

env = Environment(name='myenv')
python_packages = ['nltk', 'numpy', 'onnxruntime','flair']

for package in python_packages:
    env.python.conda_dependencies.add_pip_package(package)

inference_config = InferenceConfig(environment=env, source_directory='./source_dir', entry_script='./score.py')

In [4]:
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=0.5, memory_gb=1, auth_enabled=True
)

In [5]:
service = Model.deploy(
    ws,
    "myservice",
    [model],
    inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

print(service.get_logs())

C:\Users\User\AppData\Local\Temp\ipykernel_19604\1635410260.py:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-05-10 10:54:37+07:00 Creating Container Registry if not exists.
2023-05-10 10:54:37+07:00 Registering the environment.
2023-05-10 10:54:38+07:00 Use the existing image.
2023-05-10 10:54:38+07:00 Generating deployment configuration.
2023-05-10 10:54:39+07:00 Submitting deployment to compute.
2023-05-10 10:54:45+07:00 Checking the status of deployment myservice..
2023-05-10 10:58:21+07:00 Checking the status of inference endpoint myservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"
/bin/bash: /azureml-envs/azureml_bf99e0798ca6a8bdacfb2455262f0b28/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_bf99e0798ca6a8bdacfb2455262f0b28/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/ba

In [9]:
import requests
import json
from azureml.core import Webservice

service = Webservice(workspace=ws, name="myservice")
scoring_uri = service.scoring_uri

# If the service is authenticated, set the key or token
key, _ = service.get_keys()

# Set the appropriate headers
headers = {"Content-Type": "application/json"}
headers["Authorization"] = f"Bearer {key}"

# Make the request and display the response and logs
data = {
    "query": "I love Berlin"
}
data = json.dumps(data)
resp = requests.post(scoring_uri, data=data, headers=headers)
print(resp.text)

"Sentence[3]: \"I love Berlin\" \u2192 [\"Berlin\"/LOC]"


In [21]:

print(service.get_logs())

/bin/bash: /azureml-envs/azureml_bf99e0798ca6a8bdacfb2455262f0b28/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_bf99e0798ca6a8bdacfb2455262f0b28/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_bf99e0798ca6a8bdacfb2455262f0b28/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2023-05-10T03:57:50,453953100+00:00 - rsyslog/run 
2023-05-10T03:57:50,458919800+00:00 - gunicorn/run 
2023-05-10T03:57:50,461212200+00:00 | gunicorn/run | 
2023-05-10T03:57:50,462568900+00:00 | gunicorn/run | ###############################################
bash: /azureml-envs/azureml_bf99e0798ca6a8bdacfb2455262f0b28/lib/libtinfo.so.6: no version information available (required by bash)
2023-05-10T03:57:50,474975800+00:00 | gunicorn/run | AzureML Container Runtime Information
2023-05-10T03:57:50,478303500+00:00 | gunicorn/run | ########################################

In [8]:
print(scoring_uri)
print(headers["Authorization"])

http://7b7ab5c1-658a-4b2b-b996-9040b928619c.northeurope.azurecontainer.io/score
Bearer W6cv8QEcPcrSjFAI3ZMdnAuYOVYBzuuL
